# Imports

In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import sklearn
from tensorflow import keras
from tensorflow.keras.utils import plot_model
import gensim

# Loading Yoga Class & Pose DataFrames
These DataFrames were scraped and created during project 3. 

In [9]:
f = open("all_poses", "rb")
all_poses = pickle.load(f)
f.close()

f = open("flask_app_df", "rb")
poses_info = pickle.load(f)
f.close()

f = open("all_yoga_classes_df", "rb")
yoga_classes = pickle.load(f)
f.close()

f = open("poses_df", "rb")
class_poses = pickle.load(f)
f.close()

f = open("vinyasa_df", "rb")
more_vinyasa = pickle.load(f)
f.close()

f = open("hatha_df", "rb")
more_hatha = pickle.load(f)
f.close()

In [10]:
yoga_classes

,Title,Poses,Class Type
0,←Slow Sunday Flow y Monday early,"[Easy Pose Hands To Heart, Easy Pose Hands Int...",Vinyasa
1,←Anahata,"[Mantra Section, Thunderbolt Pose, Easy Pose B...",Vinyasa
2,←CORE,"[Classic Sun Salutation Variation F, Chair Pos...",Vinyasa
3,←,"[Easy Pose, Easy Pose Warm Up Flow, Sun Saluta...",Vinyasa
4,←Vinyasa - Bench press & push up #,"[Corpse Pose, Corpse Pose Roll Under Spine, Wr...",Vinyasa
...,...,...,...
1582,←Shoulder-Heart Opener,"[Easy Pose, Easy Pose Neck Side Stretch, Easy ...",Yin
1583,←Yin Sequence,"[Easy Pose, Cat Cow Pose, Child Pose, Seated S...",Yin
1584,←Practicing Presence,"[Easy Pose Hands To Heart, Easy Pose Twist Rai...",Yin
1585,←Yin- Cooling,"[Sitali, Bound Angle Pose Blocks, Sun Breaths,...",Yin


### Additional Classes
I scraped 20,000 additional classes since project 3, to supplement the data. In particular, they are vinyasa and hatha classes.

In [11]:
more_vinyasa

,Title,Poses,Class Type
0,←Foundations -,"[Easy Pose, Easy Pose Elbows On Floor, Easy Po...",Vinyasa
1,←Massage therapist,"[Mountain Pose, Palm Tree Pose, Palm Tree Pose...",Vinyasa
2,←VIN I/II mins,"[Easy Pose, Seated Neck Rolls, Easy Pose Neck ...",Vinyasa
3,←Core Feather,"[Mountain Pose, Mountain Pose Raised Hands Hea...",Vinyasa
4,←Vinyasa for Shoulders,"[Reclined Arms Movement Flow, Supine Snow Ange...",Vinyasa
...,...,...,...
3811,←Back and Legs Sequence,"[Child Pose, Table Top Child Pose Flow, Cat Co...",Vinyasa
3812,←Vinyasa L-Stand,"[Corpse Pose, Pranayama, Supine Windshield Wip...",Vinyasa
3813,←inver,"[Reclined Crow Pose, Reclined Eagle Crunches, ...",Vinyasa
3814,←Basics Class for Hip Opening,"[Easy Pose Block, Seated Cat Cow Pose, Mountai...",Vinyasa


In [12]:
more_hatha

,Title,Poses,Class Type
0,←Low Back & Hamstrings //,"[Seated Straddle Pose Knees Bent Bolster, Easy...",Hatha
1,←Basic Yoga Sequenz (YTT),"[Prop Strap, Prop Block, Three Part Breath Cor...",Hatha
2,←evening,"[Easy Pose, Easy Pose Variation Side Bend, Eas...",Hatha
3,←Standing class warm-up,"[Mountain Pose, Mountain Pose Namaste, Palm Tr...",Hatha
4,←Sequence #,"[Wind Release Pose, Half Wind Release Pose, Ha...",Hatha
...,...,...,...
4623,←Seri,"[Standing Cat Cow Pose, Downward Facing Dog Po...",Hatha
4624,←Flexibility I,"[Mountain Pose, Mountain Pose Raised Hands Hea...",Hatha
4625,←Postnatal-,"[Kneeling Pose, Cat Cow Pose Forearms, Hoverin...",Hatha
4626,←Herbst - Flow,"[Mountain Pose, Chair Pose Deep Bend, Standing...",Hatha


In [269]:
df = pd.concat([yoga_classes, more_vinyasa, more_hatha])

In [270]:
df.shape

(63337, 3)

### Excluding Restorative Class Types
I am excluding the classes that are of type "gentle", "restorative" and "yin", as these are very slow restorative/laying down type classes that I'm not aiming to create (yet) with this app. I want more dynamic, flow and exercise based classes to be generated, plus I have much more data availble in support of creating those types. 

In [271]:
df = df.loc[df["Class Type"].isin(["Vinyasa", "Hatha", "Power", "Iyengar", "Ashtanga"])]

In [272]:
df.reset_index(inplace=True)

In [273]:
df

,index,Title,Poses,Class Type
0,0,←Slow Sunday Flow y Monday early,"[Easy Pose Hands To Heart, Easy Pose Hands Int...",Vinyasa
1,1,←Anahata,"[Mantra Section, Thunderbolt Pose, Easy Pose B...",Vinyasa
2,2,←CORE,"[Classic Sun Salutation Variation F, Chair Pos...",Vinyasa
3,3,←,"[Easy Pose, Easy Pose Warm Up Flow, Sun Saluta...",Vinyasa
4,4,←Vinyasa - Bench press & push up #,"[Corpse Pose, Corpse Pose Roll Under Spine, Wr...",Vinyasa
...,...,...,...,...
48113,4623,←Seri,"[Standing Cat Cow Pose, Downward Facing Dog Po...",Hatha
48114,4624,←Flexibility I,"[Mountain Pose, Mountain Pose Raised Hands Hea...",Hatha
48115,4625,←Postnatal-,"[Kneeling Pose, Cat Cow Pose Forearms, Hoverin...",Hatha
48116,4626,←Herbst - Flow,"[Mountain Pose, Chair Pose Deep Bend, Standing...",Hatha


Looking at the average length of classes so I know what length of class I should typically be generating.

In [274]:
# average length of classes
lengths = []
for i in range(len(df["Poses"])):
    length = len(df["Poses"][i])
    lengths.append(length)
print(np.average(length))

33.0


# Word2Vec
I am creating word embeddings with Word2Vec via gensim. I'm training the model on the yoga classes to create vectors of the similarities between different yoga poses, based on how close together (sequentially) they're featured within the yoga classes. 

Making the class pose lists into the type of input Word2Vec is looking for:

In [275]:
documents = [class_list for class_list in df["Poses"]]
documents[0:2]

[['Easy Pose Hands To Heart',
  'Easy Pose Hands Interlocked Behind Head',
  'Seated Torso Circles',
  'Easy Pose Variation Side',
  'Half Easy Pose Half Forward Bend',
  'Seated Windshield Wiper Pose Chest Opener Variation',
  'Cat Cow Pose',
  'Child Pose Hip Circles Flow',
  'Thread The Needle Pose Flow',
  'Downward Facing Dog Pose',
  'Downward Facing Dog Pose Plank Pose Flow',
  'Right Leg Forward',
  'Three Legged Downward Facing Dog Pose Prep',
  'Fallen Triangle Pose Variation',
  'Repeat Other Side',
  'Right Leg Forward',
  'Three Legged Downward Facing Dog Pose',
  'Low Lunge Hands To Knee',
  'Low Lunge Pose Open Twist',
  'Revolved Low Lunge Pose Knee On The Floor',
  'Runners Lunge Pose',
  'Revolved Lunge Pose',
  'Extended Side Angle Pose Variation Hand In Front',
  'Three Legged Downward Facing Dog Pose',
  'Low Lunge Hands To Knee',
  'Left Leg Forward',
  'Repeat Other Side',
  'Top Of Mat Section',
  'Mountain Pose',
  'Crescent High Lunge Pose Arms On Hips Flow',


Choosing to keep poses which show up in at least 50 classes (out of 48000).

In [276]:
model = gensim.models.Word2Vec(documents, size=100, window=5, min_count=50, sg=1)

In [277]:
len(list(model.wv.vocab.items()))

2434

In [278]:
print(model["Cat Cow Pose"])

[-0.03218082 -0.1518022  -0.21900013 -0.22784647 -0.05391332  0.23906896
 -0.21320449 -0.21570173 -0.45768702 -0.1766179   0.08589573  0.11216951
  0.01759378  0.07493998 -0.15698856  0.1863491   0.20825997 -0.21320613
  0.36808664  0.46683544 -0.06278304 -0.1640251   0.19787319  0.5703424
 -0.00761932 -0.02809849  0.4987022  -0.16976891 -0.03411588 -0.12097323
 -0.2918854  -0.00962753 -0.06204242 -0.21476337  0.3436619  -0.40890104
 -0.01819395 -0.12126855  0.24949613  0.14719892 -0.4577913  -0.04162944
 -0.11526522  0.2957142   0.12739252 -0.14287552 -0.34979028  0.1284238
 -0.1259271   0.1286666   0.24198945 -0.08204111 -0.14193207  0.05506501
  0.00451074  0.6186688  -0.11915226  0.0780672   0.21961652  0.26104498
  0.05787276 -0.06865472 -0.61855805 -0.05995261  0.04905183 -0.03994423
  0.406545   -0.06811796  0.19840765  0.11315206  0.16041619 -0.02875773
 -0.07983094  0.2568253  -0.14525127  0.0757969  -0.10302401  0.15706488
 -0.08536102  0.01431387 -0.05461498  0.24123959 -0.0

In [279]:
model.most_similar("Downward Facing Dog Pose", topn=20)

[('Plank Pose', 0.7478853464126587),
 ('Downward Facing Dog Pose Plank Pose Flow', 0.6874706745147705),
 ('Runners Lunge Pose', 0.6865273714065552),
 ('Three Legged Downward Facing Dog Pose', 0.6853184700012207),
 ('Three Legged Downward Facing Dog Pose Tiger Curl Pose Flow',
  0.6817233562469482),
 ('Low Lunge Pose', 0.6691491603851318),
 ('Standing Forward Fold Pose', 0.6663710474967957),
 ('Three Legged Downward Facing Dog Pose Variation Stacked',
  0.6627469062805176),
 ('Table Top Pose', 0.6567497253417969),
 ('Downward Facing Dog Pose Variation Knees Bent', 0.6391588449478149),
 ('Revolved Lunge Pose', 0.6241694092750549),
 ('Downward Facing Dog Pose Heels To Side', 0.6126556396484375),
 ('Downward Dog Block Thighs', 0.6117755174636841),
 ('Low Lunge Pose Hands On Hips', 0.611741304397583),
 ('Four Limbed Staff Pose', 0.6101218461990356),
 ('Revolved Low Lunge Pose Knee On The Floor', 0.6075565814971924),
 ('High Lunge Pose', 0.6068105697631836),
 ('Crescent Low Lunge Pose Variat

In [280]:
model.most_similar("Corpse Pose")

[('Corpse Pose (IRT)', 0.7625349760055542),
 ('Wind Release Pose', 0.7379505038261414),
 ('Savasana Section', 0.7282826900482178),
 ('Corpse Pose Variation Bolster', 0.7136248350143433),
 ('Corpse Pose Blanket', 0.7122341394424438),
 ('Corpse Pose Roll Under Spine', 0.6960148811340332),
 ('Deep Relaxation Technique', 0.6891850829124451),
 ('Wind Release Pose Nose To Knee', 0.6836726665496826),
 ('Happy Baby Pose Bolster Under Sacrum', 0.6831930875778198),
 ('Supine Spinal Twist Pose II', 0.6725602746009827)]

Problem is a lot of the above aren't even really variations... they should probably be reduced to their base pose before applying the word embedding model. 

In [263]:
model.most_similar("Plank Pose")

[('Plank Pose Knees Bent', 0.7375870943069458),
 ('Downward Facing Dog Pose', 0.7266370058059692),
 ('Four Limbed Staff Pose', 0.7133201360702515),
 ('Plank Pose Variation Knee To Ground (Variation)', 0.7053859829902649),
 ('Downward Facing Dog Pose Plank Pose Flow', 0.7001957893371582),
 ('Plank Pose Variation Foot to Knee', 0.6751132607460022),
 ('Runners Lunge Pose', 0.6705726385116577),
 ('Three Legged Downward Facing Dog Pose', 0.6701911687850952),
 ('Plank Pose Four Limbed Staff Pose Flow', 0.6578046083450317),
 ('Four Limbed Staff Pose Knees To Floor Variation', 0.6399896740913391)]

In [264]:
model.most_similar("Wheel Pose", topn=20)

[('Wheel Pose Prep', 0.7732982635498047),
 ('One-Legged Wheel Pose', 0.7651089429855347),
 ('Two-legged Inverted Staff Pose', 0.6853083968162537),
 ('Wheel Pose Block Between Knees', 0.6792454719543457),
 ('Bridge Pose Variation Arms Support Legs Straight', 0.6351526975631714),
 ('Ear Pressure Pose', 0.6308561563491821),
 ('Plough Pose', 0.6233429312705994),
 ('Shoulderstand Pose', 0.6058846116065979),
 ('Fish Pose Variation Lotus Elbows', 0.6032376885414124),
 ('Bridge Pose Wheel Pose Flow', 0.6008906960487366),
 ('Unsupported Shoulder Stand Pose', 0.5988141298294067),
 ('Embryo Pose in Shoulder Stand', 0.5977175235748291),
 ('Half Shoulderstand Pose Bicycle Legs', 0.5955791473388672),
 ('Shoulderstand Seal Pose', 0.5900613069534302),
 ('Wheel Pose On Ball', 0.5830180644989014),
 ('Inversions', 0.5799775719642639),
 ('Bridge Pose Variation Bound Hands', 0.574094295501709),
 ('Plough Pose Hands On Back', 0.5690933465957642),
 ('One Legged Shoulderstand Pose Hands On Back', 0.5639292001

In [268]:
model.most_similar("Sun Salutation A")

[('Sun Salutation B', 0.8779213428497314),
 ('Classic Sun Salutation Variation E', 0.6588842868804932),
 ('Sun Salutation A Variation', 0.6466954946517944),
 ('Classic Sun Salutation Variation', 0.6326018571853638),
 ('Classic Sun Salutation Variation Ii', 0.6122204661369324),
 ('Classic Sun Salutation Variation Ii Second Half', 0.6038161516189575),
 ('Sun Salutation A First Half', 0.5960967540740967),
 ('Mountain Pose', 0.580011785030365),
 ('Sun Salutation', 0.576127827167511),
 ('Classic Sun Salutation Variation Ii First Half', 0.5568618774414062)]

In [265]:
print(model.similarity("Corpse Pose", "Hand to Big Toe Pose"))
print(model.similarity("Corpse Pose", "Pigeon Pose"))
print(model.similarity("Corpse Pose", "Child Pose"))
print(model.similarity("Corpse Pose", "Wind Release Pose"))

0.09997491
0.2763628
0.34287545
0.73999816


In [ ]:
model.n_similarity(["Corpse Pose, ""])

Honestly that looks great? Very low similarity between corpse pose and an intense standing/balance pose, slightly higher for an intense ground stretch, moderate similarity for another gentle pose but in a very different body orientation, and highestest for a similar supine relaxation pose. 

However, there are currently a couple thousand total pose variations, many of which aren't even really distinct poses. I set the minimum at a range of values, higher meant to exclude less common poses, but it is losing key information -- even set at just 100, I lost "pigeon pose" which is very common. This is probably because it is listed with slightly varying names across classes. Hence, instead of just deleting poses if they don't show up often enough, I want to reduce all the variations/duplicate name variations to their base poses, which will then show up far more often and be more useful.

Besides, for the practicality of my app and creating animations to accompany each pose (which I, at least for now, anticpate having to hand pose each one), I can't accomodate thousands of poses, only a few hundred. 

For that reason, I'm going to reduce the poses in each class to their base poses before doing word embeddings. 

I want to isolate just information about poses and base poses, for simplification of all the poses in classes from thousands of variations down to 

In [330]:
poses_info.head(3)

,Base Pose,Level,Pose Name,Back-Bend,Balance,Forward-Bend,Inversion,Meditative,Pranayama,Restorative,...,Standing,Supine,Toes,Crown Chakra (Sahasrara Chakra),Heart Chakra (Anahata Chakra),Root Chakra (Muladhara Chakra),Sacral Chakra (Swadisthana Chakra),Solar Plexus (Manipura Chakra),Third Eye Chakra (Ajna Chakra),Throat Chakra (Vishuddha Chakra)
0,Boat Pose,Advanced,Boat Pose Aerial,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,Boat Pose,Advanced,Boat Pose Twist Variation Hands To Feet,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Boat Pose,Advanced,Elevated Boat Pose,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [365]:
base_poses = poses_info[["Pose Name", "Base Pose"]]
base_poses

,Pose Name,Base Pose
0,Boat Pose Aerial,Boat Pose
1,Boat Pose Twist Variation Hands To Feet,Boat Pose
2,Elevated Boat Pose,Boat Pose
3,Half Bound Lotus Boat Pose,Boat Pose
4,Bicycle Pose,Boat Pose
...,...,...
3211,Half Wind Release Pose Nose To Knee Foot Raised,Wind Release Pose
3212,Hip Opening Yoga Sequence Hippie Flow,Wind Release Pose
3213,Reclined Cow Face Pose,Wind Release Pose
3214,Seated Broken Wings Pose,Wind Release Pose


In [366]:
base_poses["Base Pose"].unique()

array(['Boat Pose', 'Bound Angle Pose', 'Bow Pose', 'Bridge Pose',
       'Camel Pose', 'Cat Cow Pose', 'Cat Tail Pulling Pose',
       'Center Splits Pose', 'Chair Pose', 'Child Pose', 'Cobra Pose',
       'Corpse Pose', 'Cow Face Pose', 'Crane Pose', 'Crow Pose',
       'Cycling Pose', 'Dancer Pose', 'Dolphin Pose',
       'Downward Facing Dog Pose', 'Eagle Pose', 'Easy Pose',
       'Easy Pose Chair', 'Exercise', 'Extended Side Angle Pose',
       'Eye Exercise', 'Eye Of The Needle Pose', 'Feathered Peacock Pose',
       'Firelog Pose', 'Fish Pose', 'Flamingo Pose',
       'Foot Behind The Head Pose', 'Four Limbed Staff Pose',
       'Garland Pose', 'Gate Pose', 'Goddess Pose',
       'Half Lord Of The Fishes Pose', 'Half Moon Pose', 'Handstand Pose',
       'Happy Baby Pose', 'Head To Knee Pose', 'Headstand Pose',
       'Hero Pose', 'High Lunge Pose', 'Intense Leg Stretch Pose',
       'Intense Side Stretch Pose', 'Legs Up The Wall Pose', 'Lion Pose',
       'Lizard Pose', 'Locust

Currently there are only 102 total base poses for all 3200 poses I collected, which seems like a bit too much of a simplification. I imagine there are certain key variations that are distinct enough that I'd want to keep them -- hence have more than 102 poses to build my classes from. So, I'm going to manually go through all 3200 poses and make modification to the associated base pose, sometimes choosing to keep the distinct variations instead, using my domain knowledge as a certified yoga teacher. 

In [383]:
# Commenting this out for readability but I manually looked at every pose...

# pd.set_option("display.max_rows", 30)
# base_poses

In [386]:
# test
base_poses.loc[4, "Base Pose"] = "Cycling Pose"

In [387]:
base_poses

,Pose Name,Base Pose
0,Boat Pose Aerial,Boat Pose
1,Boat Pose Twist Variation Hands To Feet,Boat Pose
2,Elevated Boat Pose,Boat Pose
3,Half Bound Lotus Boat Pose,Boat Pose
4,Bicycle Pose,Cycling Pose
...,...,...
3211,Half Wind Release Pose Nose To Knee Foot Raised,Wind Release Pose
3212,Hip Opening Yoga Sequence Hippie Flow,Wind Release Pose
3213,Reclined Cow Face Pose,Wind Release Pose
3214,Seated Broken Wings Pose,Wind Release Pose


In [415]:
changes = [([4, 17, 47, 50, 60], "Cycling Pose"),
    ([19, 40, 41, 44, 46, 28, 54, 67, 68, 72], "High Boat To Low Boat Flow"),
    ([64, 66, 69, 70], "Low Boat Pose"),
    ([86], "Staff Pose"), 
    ([87, 88, 117, 118], "Bound Angle Forward Bend"), 
    ([125, 135, 136, 138, 140, 141], "One Legged Bow Pose"), 
    ([216, 217, 218, 219], "Pigeon Pose"), 
    ([233, 234, 235, 236, 237, 238, 239], "Thunderbolt Pose"), 
    ([248, 269, 270, 271, 274], "Cow Pose"), 
    ([259, 260, 261, 262, 263, 364, 265, 266, 267, 268, 272, 273, 275], "Cat Pose"),
    ([291], "Hurdlers Pose"),
    ([293, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 372], "Revolved Chair Pose"), 
    ([300, 303, 306, 307, 308], "Chair Pose With Airplane Arms"),
    ([365, 369, 370], "Figure Four Pose"), 
    ([368], "Shiva Squat Pose"), 
    ([520, 521], "Supine Spinal Twist Pose"), 
    ([587, 588], "Flying Pigeon Pose"), 
    ([594, 589], "Side Crow Pose"), 
    ([599], "Baby Crow Pose"), 
    ([663, 664, 665, 666, 667, 668], "One Legged Mountain Pose"), 
    ([687, 758], "One Handed Downward Facing Dog Pose"),
    ([688, 689, 690, 759, 760], "Standing Splits Pose"), 
    ([691, 692, 693, 694, 724, 735, 736, 737, 761, 762, 763, 764], "Three Legged Downward Facing Dog Pose"), 
    ([700, 717], "Downward Facing Dog Upward Facing Dog Pose Flow"), 
    ([714], "Downward Facing Dog Pose Plank Pose Flow"), 
    ([715], "Downward Facing Dog Pose Table Top Pose Flow"), 
    ([747], "Downward Facing Dog Pose Knee To Nose"),
    ([748], "Downward Facing Dog Pose Shoulder Taps"), 
    ([749], "Downward Facing Dog Pose to Low Lunge Pose Flow"), 
    ([865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901], "Easy Pose"), 
    ([924, 925, 926], "Cactus Arms"), 
    ([998], "Half Lotus Pose"), 
    ([999], "Lotus Pose"), 
    ([1016, 1017], "Eye Exercise"), 
    ([1055, 1056, 1057, 1058, 1061], "Scorpion Pose"), 
    ([1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151], "Chaturanga Dandasana"),
    ([1387], "Headstand Pose Eagle Legs"), 
    ([1390], "Headstand Pose Lotus Legs"), 
    ([1392], "Headstand Pose Wide Legs"), 
    ([1395, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407], "Tripod Headstand Pose"), 
    ([1464, 1465, 1479, 1480, 1481, 1482], "Revolved High Lunge Pose"), 
    ([1466, 1467, 1468, 1469, 1483], "Runners Lunge Pose"), 
    ([1487, 1488, 1489, 1490, 1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509, 1510, 1511, 1512, 1513, 1514, 1515, 1516, 1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524, 1525, 1526, 1527, 1528, 1529, 1530, 1531, 1532, 1533], "Wide Legged Forward Fold"), 
    ([1487, 1509, 1510, 1511, 1512], "Revolved Wide Legged Forward Fold"),
    ([1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498], "Wide Legged Forward Fold With Halfway Lift"),
    ([1489, 1490, 1527], "Five Pointed Star Pose"), 
    ([1508], "Pyramid Pose"), 
    ([1534, 1535, 1536, 1537, 1538, 1539, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1548, 1549, 1550, 1551, 1552, 1553, 1554, 1555, 1556, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564, 1565, 1566], "Pyramid Pose"), 
    ([1584], "Flying Lizard Pose"), 
    ([1673, 1684, 1685, 1591, 1700, 1708, 1710, 1711, 1712], "Revolved Low Lunge Pose"),
    ([1732, 1733, 1745, 1758, 1758, 1776, 1777, 1778, 1784, 1790, 1791, 1791], "One Legged Moutain Pose"), 
    ([1840, 1841, 1842, 1843, 1844, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1852], "Easy Pose"), 
    ([1873, 1874, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 1882], "Standing Side Bend Pose"), 
    ([1891, 1892, 1893, 1901, 1901, 1903, 1904, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922], "King Pigeon Pose"), 
    ([1927, 1928, 1932], "Grasshopper Pose"), 
    ([1929], "Dragonfly Pose"), 
    ([1930], "Eight Angle Pose"),
    ([1934, 1970, 1971, 1972], "One Legged Plank Pose"), 
    ([1939], "Side Plank Pose"), 
    ([1958, 1959, 1960, 1961, 1967, 1968, 1969], "Forearm Plank Pose"), 
    ([2173], "Lotus Pose"), 
    ([2188, 2189, 2190, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2199, 2200], "Revolved Extended Side Angle Pose"), 
    ([2189, 2195, 2196], "Revolved High Lunge Pose"), 
    ([2347, 2348, 2349, 2350, 2351, 2352, 2353, 2354, 2355, 2356, 2357, 2358, 2359, 2360, 2361, 2362, 2363, 2364, 2365, 2366, 2367, 2368], "Skandasana"),
    ([2390, 2397], "Anantasana"), 
    ([2415, 2416, 2417, 2418, 2419, 2420, 2443], "Visvamitrasana Pose"), 
    ([2422, 2427, 2428, 2430, 2431, 2433, 2453, 2458, 2460], "Side Plank Pose With Leg Variation"), 
    ([2436, 2437, 2438, 2439, 2440, 2441], "Forearm Side Plank Pose"), 
    ([2465, 2466], "Wild Thing Pose"), 
    ([2482, 2483, 2484, 2486, 2487, 2488, 2489, 2480, 2492, 2493], "Half Splits Pose"), 
    ([2494], "Standing Splits Pose"), 
    ([2592, 2593, 2594], "Firefly Pose"), 
    ([2611, 2612, 2613, 2614, 2615, 2616, 2617, 2618, 2619, 2620, 2621], "Forward Fold Pose With Halfway Lift"), 
    ([2688, 2689, 2692, 2717], "Bird Of Paradise Pose"), 
    ([2700, 2701, 2703], "Revolved Hand To Big Toe Pose"), 
    ([2750, 2758], "Half Sun Salutation"), 
    ([2759], "Second Half Of Sun Salutation"), 
    ([2771, 2772], "Banana Pose"), 
    ([2795], "Supine Spinal Twist Pose"), 
    ([2797, 2798, 2833, 2848, 2849, 2850], "Tiger Pose"), 
    ([2799, 2816, 2827], "Table Top Knee To Nose Flow"), 
    ([2800, 2817], "Child Pose Table Top Pose Flow"), 
    ([2801, 2831, 2834, 2837, 2840, 2841, 2842], "Table Top Pose With One Leg Extended Back"), 
    ([2809, 2810, 2811, 2812, 2813, 2814, 2815], "Revolved Table Top Pose"), 
    ([2852, 2853, 2854, 2855], "Table Top Balancing Pose, Opposite Arm and Leg Extended"), 
    ([2968, 2972, 2983, 2974, 2975, 2976, 2977, 3005, 3006, 3007, 3008], "Revolved Triangle Pose"), 
    ([3023, 3024, 3025, 3026, 3031, 3032, 3038, 3039, 3040], "Reverse Table Top Pose"), 
    ([3037], "Flip The Dog Pose"), 
    ([3046, 3047, 3048, 3049, 3050, 3051, 3052, 3053, 3054, 3055, 3056, 3057, 3058, 3059, 3060, 3061, 3062, 3063], "Anantasana"), 
    ([3064, 3065, 3066, 3067, 3070, 3071, 3072, 3073, 3074, 3075, 3079], "Warrior Pose I"), 
    ([3068, 3069, 3076, 3077, 3078], "Humble Warrior Pose"), 
    ([3106, 3107, 3108, 3109, 3110, 3111, 3112, 3113, 3114, 3115, 3116, 3117, 3118, 3119, 3120, 3121], "Warrior Pose II"), 
    ([3122, 3123, 3124, 3125, 3126, 3127, 3128, 3129, 3130, 3131, 3132, 3133, 3134, 3135, 3136, 3137, 3138, 3139, 3140, 3141, 3142, 3143, 3144], "Warrior Pose III"), 
    ([3134], "Shiva Squat Pose"), 
    ([3131, 3132, 3133], "Airplane Pose"), 
    ([3181, 3182, 3183, 3184, 3185, 3186, 3187, 3188, 3189, 3190], "Half Wind Release Pose")]

In [417]:
for i in changes:
    base_poses.loc[i[0], "Base Pose"] = i[1]

In [430]:
len(base_poses["Base Pose"].unique())

170

In [444]:
base_poses["Base Pose"].unique()

array(['Boat Pose', 'Cycling Pose', 'High Boat To Low Boat Flow',
       'Low Boat Pose', 'Bound Angle Pose', 'Staff Pose',
       'Bound Angle Forward Bend', 'Bow Pose', 'One Legged Bow Pose',
       'Bridge Pose', 'Camel Pose', 'Pigeon Pose', 'Thunderbolt Pose',
       'Cow Pose', 'Cat Cow Pose', 'Cat Pose', 'Cat Tail Pulling Pose',
       'Center Splits Pose', 'Hurdlers Pose', 'Revolved Chair Pose',
       'Chair Pose', 'Chair Pose With Airplane Arms', 'Figure Four Pose',
       'Shiva Squat Pose', 'Child Pose', 'Cobra Pose', 'Corpse Pose',
       'Supine Spinal Twist Pose', 'Cow Face Pose', 'Crane Pose',
       'Flying Pigeon Pose', 'Side Crow Pose', 'Baby Crow Pose',
       'Crow Pose', 'Dancer Pose', 'One Legged Mountain Pose',
       'Dolphin Pose', 'Downward Facing Dog Pose',
       'One Handed Downward Facing Dog Pose', 'Standing Splits Pose',
       'Three Legged Downward Facing Dog Pose',
       'Downward Facing Dog Upward Facing Dog Pose Flow',
       'Downward Facing Dog P

In [422]:
base_documents = []
for i in range(len(documents)):
    temp_df = pd.DataFrame(documents[i], columns=["0"])
    temp_df = pd.merge(temp_df, base_poses, how="left", left_on="0", right_on="Pose Name")
    temp_df = temp_df.dropna(how="any")
    new_doc = [pose for pose in temp_df["Base Pose"]]
    base_documents.append(new_doc)

Running again on documents with base_poses only:

In [423]:
model = gensim.models.Word2Vec(base_documents, size=100, window=5, min_count=1, sg=1)

In [424]:
model.most_similar("Downward Facing Dog Pose", topn=20)

[('Three Legged Downward Facing Dog Pose', 0.7752646207809448),
 ('Plank Pose', 0.652948260307312),
 ('Standing Forward Fold Pose', 0.6486674547195435),
 ('Forward Fold Pose With Halfway Lift', 0.6484947204589844),
 ('Low Lunge Pose', 0.6375930309295654),
 ('Chaturanga Dandasana', 0.6356512308120728),
 ('Mountain Pose', 0.6152158975601196),
 ('Table Top Pose', 0.6037572622299194),
 ('Downward Facing Dog Pose Knee To Nose', 0.6012575626373291),
 ('Downward Facing Dog Pose Plank Pose Flow', 0.5903475284576416),
 ('Upward Facing Dog Pose', 0.583499550819397),
 ('Palm Tree Pose', 0.5730133056640625),
 ('Child Pose', 0.5546844005584717),
 ('Warrior Pose I', 0.5540790557861328),
 ('Reverse Warrior Pose', 0.5399295687675476),
 ('Revolved High Lunge Pose', 0.5390284061431885),
 ('Table Top Balancing Pose, Opposite Arm and Leg Extended',
  0.5379910469055176),
 ('Warrior Pose II', 0.5379220247268677),
 ('Cobra Pose', 0.5191168189048767),
 ('One Handed Downward Facing Dog Pose', 0.51019990444183

In [425]:
model.most_similar("Corpse Pose", topn=20)

[('Wind Release Pose', 0.8253781199455261),
 ('Supine Spinal Twist Pose', 0.7935897707939148),
 ('Half Wind Release Pose', 0.7451516389846802),
 ('Happy Baby Pose', 0.737824559211731),
 ('Reclined Bound Angle Pose', 0.7172040939331055),
 ('Bridge Pose', 0.6527498364448547),
 ('Eye Of The Needle Pose', 0.6461621522903442),
 ('Banana Pose', 0.6410391330718994),
 ('Legs Up The Wall Pose', 0.6343509554862976),
 ('Cycling Pose', 0.5977532863616943),
 ('Fish Pose', 0.5841200947761536),
 ('Plough Pose', 0.5591453313827515),
 ('Reclined Big Toe Pose', 0.5579828023910522),
 ('Side Lying Corpse Pose', 0.5431381464004517),
 ('Section', 0.5370664000511169),
 ('Easy Pose', 0.48950403928756714),
 ('Pranayama', 0.4437270164489746),
 ('Neck Rolls', 0.4398112893104553),
 ('Seated Forward Bend Pose', 0.43820858001708984),
 ('High Boat To Low Boat Flow', 0.43775245547294617)]

In [426]:
model.most_similar("Wheel Pose", topn=20)

[('Shoulderstand Pose', 0.6132398247718811),
 ('Fish Pose', 0.5154325366020203),
 ('Upward Plank Pose', 0.49875175952911377),
 ('Plough Pose', 0.49492019414901733),
 ('Scorpion Pose', 0.48720359802246094),
 ('Bridge Pose', 0.48524922132492065),
 ('Reclined Hero Pose', 0.4735625684261322),
 ('Handstand Pose', 0.46006903052330017),
 ('Headstand Pose', 0.44781529903411865),
 ('Camel Pose', 0.4384971261024475),
 ('Headstand Pose Eagle Legs', 0.4381139576435089),
 ('Bow Pose', 0.43379443883895874),
 ('Headstand Pose Lotus Legs', 0.43201956152915955),
 ('Dragonfly Pose', 0.42911750078201294),
 ('Reverse Table Top Pose', 0.4210657477378845),
 ('Feathered Peacock Pose', 0.4192594885826111),
 ('Eight Angle Pose', 0.41475605964660645),
 ('Reclining Hero Pose', 0.40742501616477966),
 ('Flip The Dog Pose', 0.4003870487213135),
 ('Hurdlers Pose', 0.3981773555278778)]

In [428]:
model.most_similar("Sun Salutation")

[('Warrior Pose I', 0.5906641483306885),
 ('Half Sun Salutation', 0.5853966474533081),
 ('Warrior Pose II', 0.5793404579162598),
 ('Triangle Pose', 0.5711530447006226),
 ('Second Half Of Sun Salutation', 0.571139931678772),
 ('Mountain Pose', 0.5569466352462769),
 ('Reverse Warrior Pose', 0.5567517876625061),
 ('Chair Pose', 0.5516870617866516),
 ('Tree Pose', 0.5514235496520996),
 ('Revolved Chair Pose', 0.5280930995941162)]

In [429]:
print(model.similarity("Corpse Pose", "Extended Side Angle Pose"))
print(model.similarity("Corpse Pose", "Pigeon Pose"))
print(model.similarity("Corpse Pose", "Child Pose"))
print(model.similarity("Corpse Pose", "Wind Release Pose"))

0.13704592
0.21905862
0.3814702
0.8253781


In [431]:
model.most_similar("Wild Thing Pose")

[('Side Plank Pose', 0.6133649349212646),
 ('Side Plank Pose With Leg Variation', 0.6059179306030273),
 ('Flip The Dog Pose', 0.5696864128112793),
 ('Three Legged Downward Facing Dog Pose', 0.5621377229690552),
 ('Downward Facing Dog Pose Knee To Nose', 0.5200204849243164),
 ('Forearm Side Plank Pose', 0.48757949471473694),
 ('Visvamitrasana Pose', 0.4829522967338562),
 ('Pigeon Pose', 0.43725696206092834),
 ('Plank Pose', 0.4310494661331177),
 ('Downward Facing Dog Pose', 0.41312068700790405)]

In [432]:
model.most_similar("Anantasana")

[('Side Lying Corpse Pose', 0.49826186895370483),
 ('Reverse Table Top Pose', 0.4588111937046051),
 ('Table Top Pose With One Leg Extended Back', 0.4554680585861206),
 ('Cat Tail Pulling Pose', 0.442752480506897),
 ('Cycling Pose', 0.4293849468231201),
 ('Upward Plank Pose', 0.4155281186103821),
 ('Banana Pose', 0.40798455476760864),
 ('Forearm Side Plank Pose', 0.3616667091846466),
 ('Reclined Big Toe Pose', 0.34704896807670593),
 ('Side Plank Pose With Leg Variation', 0.3463432788848877)]

In [433]:
def analogy(worda, wordb, wordc):
    result = model.most_similar(negative=[worda], positive=[wordb, wordc])
    return result[0][0]

In [440]:
# A pose to a 3-limbed variation of the same pose
analogy("Downward Facing Dog Pose", "Three Legged Downward Facing Dog Pose", "Table Top Pose")

'Table Top Balancing Pose, Opposite Arm and Leg Extended'

In [441]:
# Two poses which often follow each other one after the other 
analogy("Warrior Pose II", "Extended Side Angle Pose", "Mountain Pose")

'Standing Forward Fold Pose'

In [443]:
analogy("Warrior Pose II", "Extended Side Angle Pose", "Warrior Pose I")

'Reverse Warrior Pose'

Analysis: Now the poses have been simplfied to their base poses and hence the total possible number of input poses my application will build yoga classes from has been reduced to 170 (from 3000+). This is much more manageable, and the custom work I did to include key additional variations provides a more robust practice. 

Given the above ratings of similarity of corpse pose to other various poses, the scores make even more sense after reducing to baes poses. Very little similarity to a difficulty strength-heavy standing pose, mild similarity to an intense prone stretch pose, moderate similarity to a relaxation prone pose, and very high similarity to wind release pose, which often tends to be practiced immediately before final savasana, corpse pose. 

I am now happy with this cleaned input data and resultant word vectors demonstrating reasonable similarity, and ready to move into LSTM training. 

# LSTM

# Wolfram Language
The Wolfram Language contains extensive info about 216 yoga poses, including muscle groups used, and the recommended Preparatory and Counter Poses. 

In [267]:
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
session = WolframLanguageSession()

In [5]:
session.evaluate(wl.WolframAlpha("Easy yoga poses"))

(Rule[(('Input', 1), 'Plaintext'), 'beginner yoga poses'],
 Rule[(('Result', 1), 'Plaintext'), "Anjaneya's Pose | Balancing Table Pose | Bharadvaja's Pose | Big Toe Pose | Bikram Triangle Pose | Boat Pose | Bound Angle Pose A | Bound Angle Pose B | Bound Angle Pose C | Cat Pose | Child's Pose | Cobra Pose | Corpse Pose | Cow Pose | Crescent High Lunge Pose | Crocodile Pose | Dead Bug Pose | Deer Pose | Diamond Pose | Diamond Stretch Pose | ... (total: 90)"],
 Rule[(('Schematic:YogaData', 1), 'Plaintext'), '\n(asymmetrical poses shown with focus on the right side of the body)'],
 Rule[(('IntensityLevelSummary:YogaData', 1), 'Plaintext'), "Anjaneya's Pose | Bharadvaja's Pose | Bound Angle Pose A | Bound Angle Pose B | Bound Angle Pose C | Cat Pose | Child's Pose | Corpse Pose | Cow Pose | Crocodile Pose | Dead Bug Pose | Deer Pose | Diamond Pose | Diamond Stretch Pose | Easy Pose | Figure Four Stretch Pose | Happy Baby Pose | Head to Knee Pose A | High Lunge Pose | Knees to Chest Pose | 

In [129]:
output = session.evaluate(wlexpr('''
    EntityClass["YogaPose", 
    "PreparatoryPoses" -> 
    MemberQ[Entity["YogaPose", "EasyPose"]]] // EntityList
    '''))
output

(Entity['YogaPose', 'AccomplishedPose'], Entity['YogaPose', 'HalfLotusPose'])

In [140]:
session.evaluate(wlexpr('''
    Entity["YogaPose", "WarriorIIIPose"]["PreparatoryPoses"]
    '''))

(Entity['YogaPose', 'IntenseSideStretchPose'],
 Entity['YogaPose', 'RevolvedTrianglePose'],
 Entity['YogaPose', 'WarriorIPose'])